In [2]:
version = "REPLACE_PACKAGE_VERSION"

# Assignment 4 - Tree-based classification; Synthesis Project

This assignment has two parts: the first part reviews tree-based classification, and the second part is a synthesis project.

In [3]:
# import some necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Suppress all warnings
import warnings
warnings.filterwarnings('ignore')

## Part 1: Classification. Predicting landscape types from satellite data.

Machine learning is being used increasingly to help fight climate change and promote sustainable land use. For this section of the assignment we will be working with a dataset derived from geospatial data, and our classification goal will be to predict the type of landscape visible by satellite. With an accurate classifier, we could monitor changes in land use and characteristics over time on a global scale.

Summary of the dataset: "The dataset is derived from two sources: 1) Landsat time-series satellite imagery from the years 2014-2015, and 2) crowdsourced georeferenced polygons with land cover labels obtained from OpenStreetMap. The crowdsourced polygons cover only a small part of the image area, and are used to extract training data from the image for classifying the rest of the image. The main challenge with the dataset is that both the imagery and the crowdsourced data contain noise (due to cloud cover in the images and inaccurate labeling/digitizing of polygons).

The 'landsat_training.csv' file contains the training data for classification. Do not use this file to evaluate classification accuracy because it contains noise (many class labeling errors).

The 'landsat_testing.csv' file contains testing data to evaluate the classification accuracy. Do not use this file to train the classifier. This file does not contain any class labeling errors.

Attribute Information (columns):

`class`: The land cover class (impervious, farm, forest, grass, orchard, water) **Note: this is the target variable to classify.**

`max_ndvi`: the maximum NDVI (normalized difference vegetation index) value derived from the time-series of satellite images.

`20150720_N - 20140101_N` : NDVI values extracted from satellite images acquired between January 2014 and July 2015, in reverse chronological order (dates given in the format yyyymmdd)."

This is a multi-class classification problem with six classes. Thus, you will want to examine both micro- and macro-averaged evaluation scores across all six classes, e.g. to see if there are landscape types that are more difficult than others to classify correctly.


Use the following code to read in the data files. Note that we convert the strings giving the landscape types (the "class" column to be predicted) to numeric labels using `LabelEncoder`.

In [4]:
df_train = pd.read_csv('assets/landsat_training.csv')
df_test  = pd.read_csv('assets/landsat_testing.csv')

from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df_train['class_code'] = enc.fit_transform(df_train['class'])
df_test['class_code']  = enc.transform(df_test['class'])

class_names = enc.classes_
print(class_names)

X_train = df_train.iloc[:,1:28]
y_train = df_train['class_code']

X_test = df_test.iloc[:,1:28]
y_test = df_test['class_code']


['farm' 'forest' 'grass' 'impervious' 'orchard' 'water']


In [5]:
y_test.head()

0    5
1    5
2    5
3    5
4    5
Name: class_code, dtype: int64

### Question 1. Basic decision trees (10 points)

Using `X_train` and `y_train` from the preceeding cell, train a `DecisionTreeClassifier` with default parameters and `random_state=0`. What are the 5 most important features found by the decision tree?

*Your function should return a tuple of length 5 of the feature names in descending order of importance.*

In [6]:
def answer_one():
    features = None
    
    from sklearn.tree import DecisionTreeClassifier
    
    clf = DecisionTreeClassifier(random_state = 0).fit(X_train, y_train)

#     feature_names = [f"feature {i}" for i in range(X_train.shape[1])]
    feature_names = X_train.columns

    importance = clf.feature_importances_
    importance

    comb = zip(feature_names,importance)
    comb = list(comb)
    comb.sort(key = lambda x:x[1], reverse = True)
    comb

    features = []
    for i in comb[:5]:
        features.append(i[0])

    features
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    return tuple(features)

In [7]:

from sklearn.tree import DecisionTreeClassifier


clf = DecisionTreeClassifier(random_state = 0).fit(X_train, y_train)

# feature_names = [f"feature {i}" for i in range(X_train.shape[1])]
feature_names = X_train.columns

importance = clf.feature_importances_
importance

comb = zip(feature_names,importance)
comb = list(comb)
comb.sort(key = lambda x:x[1], reverse = True)
comb

final = []
for i in comb[0:5]:
    final.append(i[0])
    
final

['max_ndvi', '20150125_N', '20141016_N', '20140813_N', '20140930_N']

In [8]:
df_train.head()

X_test

,max_ndvi,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,20150226_N,20150210_N,...,20140626_N,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N
0,2750.110,1361.8900,-247.508,-521.17300,-1123.910,2653.0800,-264.4140,-456.4660,2080.85000,-2041.430,...,2081.940,2380.00,1078.400,1778.030,2673.610,-178.9710,2238.5800,1000.760,-81.5418,1146.260
1,2646.250,1760.8500,481.100,6.26571,398.950,2646.2500,550.9820,92.6923,2188.03000,-1049.050,...,2178.550,2009.69,1083.700,2029.520,1033.900,-19.6148,2183.6600,927.882,668.7500,1519.740
2,3509.620,3509.6200,-241.320,-1239.32000,-330.388,1257.1900,-175.1710,-294.5000,-1315.06000,2486.500,...,2524.020,1838.51,1538.750,846.783,965.654,-468.0240,-222.3120,1124.250,1132.9000,594.059
3,2820.470,2138.6900,-126.186,-522.28100,-741.163,2334.7000,-207.4360,-76.9686,2820.47000,-1427.560,...,2195.570,2098.55,984.456,2239.330,593.874,1540.8800,2398.4200,846.470,572.3080,1311.610
4,2684.130,1995.8000,-1029.570,1713.62000,-472.258,2526.9900,-91.7429,-146.2260,2487.71000,1185.190,...,2560.820,2684.13,1432.150,2174.890,1321.690,1351.5800,2381.9500,870.890,1174.3300,991.427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,491.678,-1673.7400,-2514.480,-2451.19000,-2738.440,64.4464,-2275.0300,-2881.1000,-4738.97000,-4293.570,...,491.678,-2257.89,-2582.420,-1699.100,-2295.300,306.4830,209.4030,221.920,-4885.1500,-2630.590
296,556.093,-96.8233,-412.727,-1795.40000,-2363.820,-2168.1900,-2162.6800,-3155.7400,-4416.11000,-5648.660,...,165.140,-3991.91,-2614.910,-2109.840,-2687.180,-2853.8900,71.3402,-86.414,-5109.0100,324.637
297,987.461,-2364.6000,-155.592,-1422.09000,-1713.400,465.6220,-2230.4000,-3088.7300,-5010.32000,-4211.420,...,-694.225,-2484.50,-1756.080,-3820.430,-1988.230,433.8520,291.8000,254.548,-4259.3000,412.115
298,973.953,-3004.6300,-1217.120,180.12200,-1113.890,438.4180,-2442.5100,-3210.5600,-3237.74000,-192.857,...,556.522,-3291.49,-2018.450,-3472.650,-3109.070,511.7920,369.0970,276.948,-5574.5100,298.739


In [9]:
stu_ans = answer_one()
stu_ans

('max_ndvi', '20150125_N', '20141016_N', '20140813_N', '20140930_N')

In [10]:
# Autograder tests

stu_ans = answer_one()

assert isinstance(stu_ans, tuple), "Q1: Your function should return a tuple. "
assert len(stu_ans) == 5, "Q1: The length of your returned tuple should be 5. "
assert all([isinstance(item, str) for item in stu_ans]), "Q1: Your tuple should only contain strings. "


del stu_ans

### Question 2.  (10 points) 

One very important decision tree parameter that can have a significant effect on how much the data are overfit (or not) is the *maximum tree depth*. Increasing maximum tree depth may increase accuracy by allowing more complex decision rules involving more features, but also increases the potential for overfitting with deeper trees.

Examine the effect of maximum tree depth on training and test set accuracy for a basic decision tree on this dataset. 
Return a list of `(train_score, test_score)` tuples for each possible maximum tree depth from 1 to 10.

Use `random_state=0` for any calls involving randomization.

In [11]:
def answer_two():
    results = []
    
    for i in list(range(1,11)):
        clf = DecisionTreeClassifier(max_depth = i,random_state = 0).fit(X_train, y_train)
        tr,te = clf.score(X_train,y_train), clf.score(X_test,y_test)

        results.append((tr,te))
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    return results

In [12]:
results = []

for i in list(range(1,11)):
    clf = DecisionTreeClassifier(max_depth = i,random_state = 0).fit(X_train, y_train)
    tr,te = clf.score(X_train,y_train), clf.score(X_test,y_test)
    
    results.append((tr,te))
    
results   
    
    



[(0.725936462778568, 0.3433333333333333),
 (0.7871028923660502, 0.43333333333333335),
 (0.8349928876244666, 0.49333333333333335),
 (0.8614509246088193, 0.5433333333333333),
 (0.8744428639165481, 0.48),
 (0.8924608819345662, 0.53),
 (0.9128496917970602, 0.5566666666666666),
 (0.9354196301564722, 0.5533333333333333),
 (0.9533428165007113, 0.5566666666666666),
 (0.9664295874822191, 0.55)]

In [13]:
# Autograder tests

stu_ans = answer_two()

assert isinstance(stu_ans, list), "Q2: Your function should return a list. "
assert len(stu_ans) == 10, "Q2: The length of your returned list should be 10. "
assert all([isinstance(item, tuple) for item in stu_ans]), "Q2: Your list should only contain tuples. "

del stu_ans

Uncommenting the code below, take a look at the classification report for this GBDT model using the *test* set to evaluate. It is instructive to compare these results with the results from previous classifiers. 

In [14]:
# Remember to comment them out before submitting the notebook

# test_accs = [t for _, t in answer_two()]
# best_tree_model = tree.DecisionTreeClassifier(random_state=0, max_depth=np.argmax(test_accs) + 1).fit(X_train, y_train)
# y_pred = best_tree_model.predict(X_test)
# print(classification_report(y_test, y_pred, target_names=class_names))
# del test_accs, best_tree_model, y_pred

### Question 3. Gradient-boosted decision trees (10 points)

We're now going to apply GBDT to the same dataset to compare its performance with ordinary decision trees. Train a default GBDT and output the training and test scores (accuracy) in that order, as a float 2-tuple. (Use `random_state=0` for any randomized operations.)

In [15]:
def answer_three():
    results = None
    
    clf = GradientBoostingClassifier(random_state = 0).fit(X_train, y_train)

    tr, te = clf.score(X_train, y_train), clf.score(X_test, y_test)

    results = tr, te
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    return results

In [16]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state = 0).fit(X_train, y_train)

tr, te = clf.score(X_train, y_train), clf.score(X_test, y_test)

tr, te



(0.9793266951161688, 0.6233333333333333)

In [17]:
#Autograder tests

stu_ans = answer_three()

assert isinstance(stu_ans, tuple), "Q3a: Your function should return a tuple. "
assert len(stu_ans) == 2, "Q3a: The length of your returned tuple should be 2. "
assert all([isinstance(item, float) for item in stu_ans]), "Q3a: Your tuple should only contain floats. "

del stu_ans

## Part 2: Synthesis project

### Question 1. City of Detroit Prediction Problem (70 points)

The synthesis project is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - building deterioration. [Blight violations](https://detroitmi.gov/departments/department-appeals-and-hearings/blight-ticket-information) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. Some related datasets of interest include:

* [Building Permits](https://data.detroitmi.gov/datasets/building-permits)
* [Trades Permits](https://data.detroitmi.gov/datasets/trades-permits)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/datasets/improve-detroit-issues)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/datasets/dpd-citizen-complaints)
* [Parcel Data](https://data.detroitmi.gov/datasets/parcels-2)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will receive full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [18]:
def blight_model():
    rec = None
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import OneHotEncoder


    #import dataframes

    train_df = pd.read_csv('assets/train.csv')
    test_df  = pd.read_csv('assets/test.csv')
    addresses_df = pd.read_csv('assets/addresses.csv')
    latlong_df = pd.read_csv('assets/latlons.csv')


    #drop cases where outcome was Null

    train_df = train_df[train_df['compliance'].notna()]
    train_df 

    ###get only features we care about training

    train_df = train_df[['agency_name','judgment_amount','fine_amount','discount_amount','clean_up_cost','compliance']].reset_index()

    #one-hot encode agency_name


    enc = OneHotEncoder(handle_unknown = 'error')
    enc.fit(train_df[['agency_name']])

    enc_df = pd.DataFrame(enc.transform(train_df[['agency_name']]).toarray())

    train_df = train_df.join(enc_df)

    train_df.drop(['index','agency_name'],axis = 1, inplace = True)

    #define X_train, y_train
    y_train = train_df['compliance']

    train_df.drop(['compliance'],axis = 1, inplace = True)

    X_train = train_df

    ###get only features we care about testing

    test_df = test_df[['ticket_id','agency_name','judgment_amount','fine_amount','discount_amount','clean_up_cost']].reset_index()

    #one-hot encode agency_name

    # enc = OneHotEncoder(handle_unknown = 'error')

    enc_df = pd.DataFrame(enc.transform(test_df[['agency_name']]).toarray())

    test_df = test_df.join(enc_df)

    ticket_ids = list(test_df['ticket_id'])

    test_df.drop(['index','agency_name','ticket_id'],axis = 1, inplace = True)


    X_test = test_df

    logreg = LogisticRegression().fit(X_train, y_train)

    preds = logreg.predict_proba(X_test)

    preds_paid = []

    for i in preds:
        preds_paid.append(i[1])

    preds_paid

    zipped = zip(ticket_ids,preds_paid)

    final = pd.DataFrame(zipped).set_index(0)

    rec = pd.Series(final[1])
    

    
    # YOUR CODE HERE
#     raise NotImplementedError()
    
    return rec

In [19]:
#import dataframes

train_df = pd.read_csv('assets/train.csv')
test_df  = pd.read_csv('assets/test.csv')
addresses_df = pd.read_csv('assets/addresses.csv')
latlong_df = pd.read_csv('assets/latlons.csv')



In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score


#import dataframes

train_df = pd.read_csv('assets/train.csv')
test_df  = pd.read_csv('assets/test.csv')
addresses_df = pd.read_csv('assets/addresses.csv')
latlong_df = pd.read_csv('assets/latlons.csv')


#drop cases where outcome was Null

train_df = train_df[train_df['compliance'].notna()]
train_df 

###get only features we care about training

train_df = train_df[['agency_name','judgment_amount','fine_amount','discount_amount','clean_up_cost','compliance']].reset_index()

#one-hot encode agency_name


enc = OneHotEncoder(handle_unknown = 'error')
enc.fit(train_df[['agency_name']])

enc_df = pd.DataFrame(enc.transform(train_df[['agency_name']]).toarray())

train_df = train_df.join(enc_df)

train_df.drop(['index','agency_name'],axis = 1, inplace = True)

#define X_train, y_train
y_train = train_df['compliance']

train_df.drop(['compliance'],axis = 1, inplace = True)

X_train = train_df

###get only features we care about testing

test_df = test_df[['ticket_id','agency_name','judgment_amount','fine_amount','discount_amount','clean_up_cost']].reset_index()

#one-hot encode agency_name

# enc = OneHotEncoder(handle_unknown = 'error')

enc_df = pd.DataFrame(enc.transform(test_df[['agency_name']]).toarray())

test_df = test_df.join(enc_df)

ticket_ids = list(test_df['ticket_id'])

test_df.drop(['index','agency_name','ticket_id'],axis = 1, inplace = True)


X_test = test_df

logreg = LogisticRegression().fit(X_train, y_train)

preds = logreg.predict_proba(X_test)

preds_paid = []

for i in preds:
    preds_paid.append(i[1])
    
preds_paid

zipped = zip(ticket_ids,preds_paid)

final = pd.DataFrame(zipped).set_index(0)

final_s = pd.Series(final[1])
final_s


NameError: name 'pd' is not defined

In [ ]:
# Autograder tests - sanity checks; no points for passing this cell

stu_ans = blight_model()

assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series. "
assert len(stu_ans) == 61001, "Your series is of incorrect length. "


In [ ]:
stu_ans = blight_model()
stu_ans

In [ ]:
# Hidden autograder tests - whether AUC >= 0.55


In [ ]:
# Hidden autograder tests - whether AUC >= 0.6


In [ ]:
# Hidden autograder tests - whether AUC >= 0.65


In [ ]:
# Hidden autograder tests - whether AUC >= 0.7


In [ ]:
# Hidden autograder tests - whether AUC >= 0.75
